In [1]:
import torchrl
from torchrl.utils import make_parallel_env

import torch
import numpy as np

from torchrl.a2c import RolloutStorage
from torchrl.a2c import ActorCritic
from collections import deque

import time
%matplotlib notebook
import matplotlib.pyplot as plt

import pdb

In [2]:
torch.set_num_threads(1)
device = torch.device("cuda")

In [3]:
params = {}
params["rollout_len"] = 5
params["num_workers"] = 32

env = make_parallel_env("BreakoutNoFrameskip-v4", 
                        seed=99, num_workers=params["num_workers"], 
                        num_frame_stack=4, device=device, log_dir="results")

params["obs_shape"] = env.observation_space.shape
params["action_shape"] = env.action_space.n

params["total_steps"] = 1e7
params["discount_gamma"] = 0.99
params["gae_tau"] = 0.95
params["use_gae"] = False
params["entropy_coef"] = 0.01
params["value_coef"] = 0.5
params['num_in_channels'] = 3
params["max_grad_norm"] = 0.5
params["learning_rate"] = 7e-4
params["RMS_alpha"] = 0.99
params["RMS_eps"] = 1e-3
params["resume"] = False
params["rnn_size"] = 512

In [5]:
obs = env.reset()

In [6]:
rollouts = RolloutStorage(params)
rollouts.obs[0].copy_(obs)
rollouts.to(device)

In [7]:
model = ActorCritic(params).to(device)

In [8]:
optimizer = torch.optim.RMSprop(model.parameters(),
                           params["learning_rate"],
                           alpha=params["RMS_alpha"],
                           eps=params["RMS_eps"]
                          )

In [9]:
num_steps = 0

In [10]:
episode_rewards = deque(maxlen=50)

t0 = time.time()

batch_size = int(params["num_workers"] * params["rollout_len"])
num_iter = int(params["total_steps"] // batch_size)
log_interval = 1000
log_iter = int(log_interval // batch_size)

rollouts.obs[0].copy_(obs)

for e in range(num_iter):
    #-------------------
    # Evaluation Network 
    #-------------------

    # collect rollout
    for step in range(params["rollout_len"]):
        # Sample actions
        with torch.no_grad():
            actions, values, rnn_hxs = model.act(rollouts.obs[step], 
                                                 rollouts.masks[step], 
                                                 rollouts.rnn_hxs[step])

        # move a step
        states, rewards, dones, infos = env.step(actions)

        for info in infos:
            if 'episode' in info.keys():
                episode_rewards.append(info['episode']['r'])

        rollouts.insert(states, rewards, 1 - torch.from_numpy(dones.astype(np.float32)), 
                        actions, values.squeeze(), rnn_hxs)

    # calculate the next value
    with torch.no_grad():
        next_value = model.get_value(rollouts.obs[-1],
                                    rollouts.masks[-1],
                                    rollouts.rnn_hxs[-1]).squeeze()

    rollouts.compute_returns(next_value)
    # to cpu or gpu
    rollouts.to(device)

    #---------------
    # Update Network 
    #---------------
    
    batch_states = rollouts.obs[:-1].view(-1, *params["obs_shape"])
    batch_actions = rollouts.actions.view(-1)
    batch_masks = rollouts.masks[:-1].view(-1)
    # only using the first rnn hidden state
    batch_rnn_hxs = rollouts.rnn_hxs[0].view(-1, params["rnn_size"])

    log_probs, values, entropy = model.eval_action(batch_states, 
                                                   batch_actions,
                                                   batch_masks,
                                                   batch_rnn_hxs)
    
    advs = rollouts.returns[:-1].view(-1) - values.squeeze()
    value_loss = advs.pow(2).mean()
    policy_loss = -(advs.detach() * log_probs).mean()
    
    loss = policy_loss + params["value_coef"]*value_loss - params["entropy_coef"]*entropy


    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(
            model.parameters(), params["max_grad_norm"])
    optimizer.step()
    
    rollouts.after_update()

    # time for logging
    if e % log_iter == 0:
        if len(episode_rewards) > 0:
            print('Total steps: %d Returns %.2f/%.2f/%.2f/%.2f (mean/median/min/max)  %.2f steps/s' % (
                num_steps,
                np.mean(episode_rewards),
                np.median(episode_rewards),
                np.min(episode_rewards),
                np.max(episode_rewards),
                batch_size * log_iter / (time.time() - t0)
            ))
            t0 = time.time()
    
    num_steps += params["rollout_len"] * params["num_workers"]


Total steps: 4800 Returns 0.00/0.00/0.00/0.00 (mean/median/min/max)  276.41 steps/s
Total steps: 5760 Returns 0.27/0.00/0.00/1.00 (mean/median/min/max)  2064.08 steps/s
Total steps: 6720 Returns 0.61/1.00/0.00/2.00 (mean/median/min/max)  1967.26 steps/s
Total steps: 7680 Returns 0.96/1.00/0.00/3.00 (mean/median/min/max)  2210.71 steps/s
Total steps: 8640 Returns 1.12/1.00/0.00/3.00 (mean/median/min/max)  2279.91 steps/s
Total steps: 9600 Returns 1.06/1.00/0.00/4.00 (mean/median/min/max)  2058.85 steps/s
Total steps: 10560 Returns 1.13/1.00/0.00/4.00 (mean/median/min/max)  2187.26 steps/s
Total steps: 11520 Returns 1.00/0.00/0.00/4.00 (mean/median/min/max)  2047.26 steps/s
Total steps: 12480 Returns 0.98/0.00/0.00/4.00 (mean/median/min/max)  2239.63 steps/s
Total steps: 13440 Returns 1.18/1.00/0.00/5.00 (mean/median/min/max)  2100.69 steps/s
Total steps: 14400 Returns 1.30/1.00/0.00/5.00 (mean/median/min/max)  2151.21 steps/s
Total steps: 15360 Returns 1.40/1.00/0.00/5.00 (mean/median/m

Total steps: 96960 Returns 1.26/1.00/0.00/7.00 (mean/median/min/max)  2198.94 steps/s
Total steps: 97920 Returns 1.30/1.00/0.00/7.00 (mean/median/min/max)  2134.26 steps/s
Total steps: 98880 Returns 1.40/1.00/0.00/7.00 (mean/median/min/max)  2178.15 steps/s
Total steps: 99840 Returns 1.50/1.00/0.00/9.00 (mean/median/min/max)  2178.02 steps/s
Total steps: 100800 Returns 1.82/1.00/0.00/9.00 (mean/median/min/max)  2149.92 steps/s
Total steps: 101760 Returns 2.00/2.00/0.00/9.00 (mean/median/min/max)  2222.52 steps/s
Total steps: 102720 Returns 2.06/2.00/0.00/9.00 (mean/median/min/max)  2133.53 steps/s
Total steps: 103680 Returns 2.02/2.00/0.00/9.00 (mean/median/min/max)  2073.66 steps/s
Total steps: 104640 Returns 2.12/2.00/0.00/9.00 (mean/median/min/max)  2234.61 steps/s
Total steps: 105600 Returns 2.14/2.00/0.00/9.00 (mean/median/min/max)  2134.44 steps/s
Total steps: 106560 Returns 2.28/2.00/0.00/9.00 (mean/median/min/max)  2146.49 steps/s
Total steps: 107520 Returns 2.36/2.00/0.00/9.00

Total steps: 188160 Returns 1.18/1.00/0.00/4.00 (mean/median/min/max)  2108.97 steps/s
Total steps: 189120 Returns 1.14/1.00/0.00/4.00 (mean/median/min/max)  2125.20 steps/s
Total steps: 190080 Returns 1.12/1.00/0.00/4.00 (mean/median/min/max)  2138.83 steps/s
Total steps: 191040 Returns 1.10/1.00/0.00/4.00 (mean/median/min/max)  2183.31 steps/s
Total steps: 192000 Returns 0.88/1.00/0.00/4.00 (mean/median/min/max)  2029.60 steps/s
Total steps: 192960 Returns 0.86/1.00/0.00/4.00 (mean/median/min/max)  2120.78 steps/s
Total steps: 193920 Returns 0.84/1.00/0.00/3.00 (mean/median/min/max)  2164.51 steps/s
Total steps: 194880 Returns 0.88/1.00/0.00/5.00 (mean/median/min/max)  2170.96 steps/s
Total steps: 195840 Returns 0.92/1.00/0.00/5.00 (mean/median/min/max)  2207.45 steps/s
Total steps: 196800 Returns 0.94/1.00/0.00/5.00 (mean/median/min/max)  2117.74 steps/s
Total steps: 197760 Returns 1.16/1.00/0.00/6.00 (mean/median/min/max)  1964.97 steps/s
Total steps: 198720 Returns 1.22/1.00/0.00/

Total steps: 278400 Returns 1.38/1.00/0.00/5.00 (mean/median/min/max)  2317.03 steps/s
Total steps: 279360 Returns 1.32/1.00/0.00/5.00 (mean/median/min/max)  2252.81 steps/s
Total steps: 280320 Returns 1.54/1.00/0.00/9.00 (mean/median/min/max)  2166.21 steps/s
Total steps: 281280 Returns 1.52/1.00/0.00/9.00 (mean/median/min/max)  2207.31 steps/s
Total steps: 282240 Returns 1.48/1.00/0.00/9.00 (mean/median/min/max)  2284.35 steps/s
Total steps: 283200 Returns 1.50/1.00/0.00/9.00 (mean/median/min/max)  2266.83 steps/s
Total steps: 284160 Returns 1.42/1.00/0.00/9.00 (mean/median/min/max)  2222.45 steps/s
Total steps: 285120 Returns 1.32/1.00/0.00/9.00 (mean/median/min/max)  2122.94 steps/s
Total steps: 286080 Returns 1.20/0.50/0.00/9.00 (mean/median/min/max)  2158.34 steps/s
Total steps: 287040 Returns 1.24/0.50/0.00/9.00 (mean/median/min/max)  2174.73 steps/s
Total steps: 288000 Returns 1.28/1.00/0.00/9.00 (mean/median/min/max)  2214.94 steps/s
Total steps: 288960 Returns 1.32/1.00/0.00/

Total steps: 368640 Returns 1.54/1.00/0.00/9.00 (mean/median/min/max)  2173.50 steps/s
Total steps: 369600 Returns 1.46/1.00/0.00/9.00 (mean/median/min/max)  2223.21 steps/s
Total steps: 370560 Returns 1.40/1.00/0.00/9.00 (mean/median/min/max)  2147.25 steps/s
Total steps: 371520 Returns 1.40/1.00/0.00/9.00 (mean/median/min/max)  2064.58 steps/s
Total steps: 372480 Returns 1.46/1.00/0.00/9.00 (mean/median/min/max)  2179.17 steps/s
Total steps: 373440 Returns 1.52/1.00/0.00/9.00 (mean/median/min/max)  2164.27 steps/s
Total steps: 374400 Returns 1.48/1.00/0.00/9.00 (mean/median/min/max)  2147.03 steps/s
Total steps: 375360 Returns 1.62/1.00/0.00/9.00 (mean/median/min/max)  2154.82 steps/s
Total steps: 376320 Returns 1.54/1.00/0.00/9.00 (mean/median/min/max)  2131.54 steps/s
Total steps: 377280 Returns 1.44/1.00/0.00/6.00 (mean/median/min/max)  2024.09 steps/s
Total steps: 378240 Returns 1.54/1.00/0.00/6.00 (mean/median/min/max)  2144.77 steps/s
Total steps: 379200 Returns 1.54/1.00/0.00/

...

Total steps: 9652800 Returns 69.38/13.50/3.00/313.00 (mean/median/min/max)  2158.55 steps/s
Total steps: 9653760 Returns 72.42/14.00/3.00/313.00 (mean/median/min/max)  2013.11 steps/s
Total steps: 9654720 Returns 78.28/14.00/3.00/332.00 (mean/median/min/max)  1992.92 steps/s
Total steps: 9655680 Returns 78.74/13.50/4.00/332.00 (mean/median/min/max)  2008.31 steps/s
Total steps: 9656640 Returns 72.42/12.50/4.00/332.00 (mean/median/min/max)  2060.80 steps/s
Total steps: 9657600 Returns 78.40/13.50/4.00/332.00 (mean/median/min/max)  2185.99 steps/s
Total steps: 9658560 Returns 79.04/14.00/4.00/332.00 (mean/median/min/max)  2135.98 steps/s
Total steps: 9659520 Returns 72.88/13.50/4.00/332.00 (mean/median/min/max)  2152.91 steps/s
Total steps: 9660480 Returns 72.96/13.50/4.00/332.00 (mean/median/min/max)  2321.10 steps/s
Total steps: 9661440 Returns 66.80/12.50/4.00/332.00 (mean/median/min/max)  2267.59 steps/s
Total steps: 9662400 Returns 66.80/12.50/4.00/332.00 (mean/median/min/max)  2356

Total steps: 9739200 Returns 69.52/12.00/1.00/375.00 (mean/median/min/max)  2200.92 steps/s
Total steps: 9740160 Returns 75.26/12.00/1.00/375.00 (mean/median/min/max)  2038.30 steps/s
Total steps: 9741120 Returns 76.14/13.00/1.00/375.00 (mean/median/min/max)  2071.24 steps/s
Total steps: 9742080 Returns 74.78/13.00/1.00/375.00 (mean/median/min/max)  2045.43 steps/s
Total steps: 9743040 Returns 74.64/13.00/1.00/375.00 (mean/median/min/max)  2279.12 steps/s
Total steps: 9744000 Returns 74.66/13.00/1.00/375.00 (mean/median/min/max)  2151.77 steps/s
Total steps: 9744960 Returns 74.66/13.00/1.00/375.00 (mean/median/min/max)  2120.00 steps/s
Total steps: 9745920 Returns 75.00/13.00/1.00/375.00 (mean/median/min/max)  2120.25 steps/s
Total steps: 9746880 Returns 74.68/13.00/1.00/375.00 (mean/median/min/max)  2270.44 steps/s
Total steps: 9747840 Returns 68.54/12.50/1.00/375.00 (mean/median/min/max)  2183.46 steps/s
Total steps: 9748800 Returns 68.54/12.50/1.00/375.00 (mean/median/min/max)  2415

Total steps: 9825600 Returns 45.62/10.00/2.00/332.00 (mean/median/min/max)  2313.45 steps/s
Total steps: 9826560 Returns 45.62/10.00/2.00/332.00 (mean/median/min/max)  2264.10 steps/s
Total steps: 9827520 Returns 45.88/11.00/2.00/332.00 (mean/median/min/max)  2118.80 steps/s
Total steps: 9828480 Returns 45.96/11.00/2.00/332.00 (mean/median/min/max)  2276.58 steps/s
Total steps: 9829440 Returns 52.54/11.00/2.00/332.00 (mean/median/min/max)  2289.28 steps/s
Total steps: 9830400 Returns 51.68/11.00/2.00/332.00 (mean/median/min/max)  2204.22 steps/s
Total steps: 9831360 Returns 51.46/10.50/2.00/332.00 (mean/median/min/max)  2318.46 steps/s
Total steps: 9832320 Returns 53.82/10.50/2.00/347.00 (mean/median/min/max)  2318.91 steps/s
Total steps: 9833280 Returns 53.36/11.00/2.00/347.00 (mean/median/min/max)  2211.72 steps/s
Total steps: 9834240 Returns 54.40/11.00/2.00/347.00 (mean/median/min/max)  2219.53 steps/s
Total steps: 9835200 Returns 49.92/10.50/2.00/347.00 (mean/median/min/max)  2285

Total steps: 9912000 Returns 42.86/8.00/2.00/332.00 (mean/median/min/max)  2279.17 steps/s
Total steps: 9912960 Returns 42.62/9.00/2.00/332.00 (mean/median/min/max)  2236.85 steps/s
Total steps: 9913920 Returns 36.80/10.00/2.00/332.00 (mean/median/min/max)  2181.85 steps/s
Total steps: 9914880 Returns 37.12/10.00/2.00/332.00 (mean/median/min/max)  2196.15 steps/s
Total steps: 9915840 Returns 37.50/10.50/2.00/332.00 (mean/median/min/max)  2304.42 steps/s
Total steps: 9916800 Returns 38.06/11.00/2.00/332.00 (mean/median/min/max)  2265.98 steps/s
Total steps: 9917760 Returns 39.14/11.00/2.00/332.00 (mean/median/min/max)  2186.23 steps/s
Total steps: 9918720 Returns 40.52/11.00/2.00/332.00 (mean/median/min/max)  2095.38 steps/s
Total steps: 9919680 Returns 34.78/11.50/2.00/332.00 (mean/median/min/max)  2193.24 steps/s
Total steps: 9920640 Returns 34.56/11.00/2.00/332.00 (mean/median/min/max)  2288.89 steps/s
Total steps: 9921600 Returns 40.42/11.00/2.00/332.00 (mean/median/min/max)  2255.3

Total steps: 9998400 Returns 25.06/9.50/3.00/332.00 (mean/median/min/max)  2252.95 steps/s
Total steps: 9999360 Returns 25.44/10.00/3.00/332.00 (mean/median/min/max)  2240.45 steps/s
